
# Análisis RFM - Entrega 1

- **Definición del problema**
- **Diccionario de datos**
- **Script inicial para leer datos y preparar métricas RFM**
---


## 1. Definición del problema

El objetivo es **segmentar clientes** utilizando el método **RFM (Recency, Frequency, Monetary)** con base en un conjunto de transacciones históricas.

- **Recency (R)**: Días desde la última compra.
- **Frequency (F)**: Número total de compras.
- **Monetary (M)**: Monto total gastado.

El análisis permitirá clasificar a los clientes en diferentes segmentos  y generar recomendaciones personalizadas.

**Entradas**:
- Archivo CSV con transacciones de clientes.

**Salidas**:
- Tabla con métricas RFM por cliente.
- Visualizaciones básicas (histogramas, ranking).

---



## 2. Diccionario de datos

| Columna      | Tipo     | Descripción |
|--------------|----------|-------------|
| CustomerID   | String   | Identificador único del cliente (ej. C001) |
| InvoiceNo    | Integer  | Número de factura o transacción |
| InvoiceDate  | Date     | Fecha de la transacción (YYYY-MM-DD) |
| Amount       | Float    | Valor monetario de la transacción |

---


##3. Script inicial

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ruta del archivo CSV (ajusta si es necesario)
csv_file = "transactions_rfm.csv"

# Cargar datos
df = pd.read_csv(csv_file)

# Convertir columna InvoiceDate a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Mostrar primeras filas y estructura
print("Dimensiones del dataset:", df.shape)
print("\nPrimeras filas:")
print(df.head())

In [ ]:
# Preparar cálculo RFM
# Fecha de referencia (fecha actual)
fecha_referencia = pd.Timestamp.today()

# 1. Calcular la fecha de la última compra por cliente
last_purchase = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
last_purchase['Recency'] = (fecha_referencia - last_purchase['InvoiceDate']).dt.days

print("=== Última compra y Recency ===")
print(last_purchase.head(), "\n")

In [ ]:
# 2. Calcular la frecuencia (cantidad de facturas por cliente)
frequency = df.groupby('CustomerID')['InvoiceNo'].count().reset_index()
frequency.rename(columns={'InvoiceNo': 'Frequency'}, inplace=True)

print("=== Frecuencia por cliente ===")
print(frequency.head(), "\n")

In [ ]:
# 3. Calcular el valor monetario (suma de Amount por cliente)
monetary = df.groupby('CustomerID')['Amount'].sum().reset_index()
monetary.rename(columns={'Amount': 'Monetary'}, inplace=True)

print("=== Valor Monetario por cliente ===")
print(monetary.head(), "\n")

In [ ]:
# 4. Unir todo en un solo DataFrame
rfm = last_purchase[['CustomerID', 'Recency']].merge(frequency, on='CustomerID').merge(monetary, on='CustomerID')

print("=== Matriz RFM final ===")
print(rfm.head())

In [ ]:
# Histograma Recency
plt.figure(figsize=(8, 5))
plt.hist(rfm['Recency'], bins=15, color='#1f77b4', edgecolor='black')
plt.title('Distribución de Recency')
plt.xlabel('Días desde última compra')
plt.ylabel('Número de clientes')
plt.show()

# Histograma Frequency
plt.figure(figsize=(8, 5))
plt.hist(rfm['Frequency'], bins=15, color='#ff7f0e', edgecolor='black')
plt.title('Distribución de Frequency')
plt.xlabel('Número de compras')
plt.ylabel('Número de clientes')
plt.show()

# Histograma Monetary
plt.figure(figsize=(8, 5))
plt.hist(rfm['Monetary'], bins=15, color='#2ca02c', edgecolor='black')
plt.title('Distribución de Monetary')
plt.xlabel('Monto total gastado')
plt.ylabel('Número de clientes')
plt.show()

# Ranking de usuarios que mas han gastado
top_10 = rfm.sort_values(by='Monetary', ascending=False).head(10)

plt.figure(figsize=(8, 5))
plt.bar(top_10['CustomerID'], top_10['Monetary'], color='#9467bd', edgecolor='black')
plt.title('Top 10 clientes por gasto total')
plt.xlabel('CustomerID')
plt.ylabel('Monto total')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Mostrar también el ranking en tabla
print("=== Top 10 clientes por valor monetario ===")
print(top_10)


---
#Analisis RFM - Entrega 2

##Novedades
- **Nuevas columnas y variables**
- **Propuesta de ML**
- **Dataset de mejor calidad**
- **Script mejorado**
---

##1. Nuevas columnas y variables

###Columnas

- **cutomer_id**: Identificador del comprador.
- **order_id**: Identificador de la factura/transaccion.
- **order_date**: Fecha de la factura/transaccion.
- **total_amopunt**: Total de la compra.
- **category (columna categórica)**:Categoria de la compra (suponiendo que una factura/transaccion solo tiene una categoria).
- **returned**: Si el producto de devolvio o no.
- **profit_margin**: Margen de ganancia por la compra.
<br>

###Variables calculadas

- **RFM**
  - **Recency**: fecha actual – última compra (días).
  - **Frequency**: número de facturas por cliente.
  - **Monetary**: gasto total.
<br>

- **Derivadas de gasto**
  - **Ticket promedio**: Monetary / Frequency.
  - **Velocidad de gasto**: Monetary / Recency.
<br>

- **Derivadas de tiempo**
  - **Antigüedad del cliente**: fecha actual – primera compra (días).
  - **Porcentaje de meses activos**: meses con ≥1 compra / meses totales desde alta.
<br>

- **Variables de lealtad**
  - **Ratio de repetición**: (Frequency – 1) / Frequency.
<br>

- **Variables de categorias**
  - **Producto favorito**: categoría más frecuente por cliente.
  - **Concentración de gasto (Herfindahl)** = ∑(pᵢ²), con pᵢ = % gasto en categoría i.
<br>

- **Variables de calidad de cliente**
  - **Porcentaje de devoluciones**:  pedidos devueltos / total pedidos.
  - **Margen medio por cliente**: promedio de profit_margin sobre sus compras.

---



##2. Propuesta ML

- **K-Means**: Asume que los datos se pueden dividir en k grupos, cada uno alrededor de un centroide (punto medio). Itera hasta que cada punto está más cerca de un centroide que de los demás.<br>Se usara como guia base.
<br>

- **GMM (Gaussian Mixture Models)**: Supone que los datos son una mezcla de distribuciones gaussianas (campanas de Gauss). Usa EM (Expectation-Maximization) para ajustar esas distribuciones y asignar probabilidades a cada punto de pertenecer a cada cluster. <br> Se realizan clusters mas realistas y maneja mejor los datos sesgados.
<br>

- **DBSCAN (Density-Based Spatial Clustering of Applications with Noise)**: Agrupa puntos según densidad. Define dos parámetros:

  - **eps**: radio máximo de vecindad.

  - **min_samples**: número mínimo de puntos para considerar un “núcleo”.

  - Crea clusters si hay suficiente densidad; los puntos que no encajan son valores atipicos (outliers).

  Podria ayudar a detectar clientes VIP ya que no son tan comunes en un ecommerce.
  
---

##3. Dataset

Este data set es ficticio y representa ventas de un ecommerce, tiene 34.500 filas y 17 columnas. Fue diseñado cuidadosamente para simular de manera realista las ventas de un tienda virtual

Resumen de columnas

- order_id: Identificador único de cada pedido
- customer_id: Identificador único de cada cliente
- product_id: Identificador único de cada producto
- category: Categoría del producto (Electrónica, Moda, Hogar, - Belleza, Deportes, Juguetes, Abarrotes)
- price: Precio unitario del producto
- discount: Descuento aplicado (%)
- quantity: Número de artículos comprados
- payment_method: Tipo de pago (Tarjeta de Crédito, Tarjeta de - Débito, UPI, PayPal, Contraentrega, Billetera)
- order_date: Fecha de la compra
- delivery_time_days: Días que tardó en entregarse el pedido
- region: Región geográfica del cliente
- returned: Si el producto fue devuelto (Sí/No)
- total_amount: Monto final de la factura después de descuentos
- shipping_cost: Costos de envío
- profit_margin: Ganancia obtenida del pedido
- customer_age: Edad del cliente (18–70)

[Link Dataset Kaggle](https://www.kaggle.com/datasets/miadul/e-commerce-sales-transactions-dataset?resource=download)

---

##4. Script mejorado
---

###4.1. Calculo de RFM y otras metricas


- Genera métricas adicionales: ticket promedio, velocidad de - gasto, antigüedad, porcentaje de meses activos, ratio de repetición.
- Usa categorías para producto favorito y concentración de gasto.
- Deriva porcentaje de devoluciones y margen medio.
- Aplica log-transformación a variables sesgadas.
- Escala todas las numéricas con StandardScaler.
- Codifica producto_favorito con One-Hot.
- Devuelve un dataset listo (sales_rfm_final) para clustering con - K-Means, GMM o DBSCAN
---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from datetime import datetime

# ==========================
# 1. Cargar dataset
# ==========================
sales_df = pd.read_csv("orders.csv", parse_dates=["order_date"])
print("\nPrimeras filas:")
print(sales_df.head())

fecha_ref = pd.Timestamp.today()

In [ ]:
# ==========================
# 2. Variables RFM
# ==========================
sales_rfm = sales_df.groupby("customer_id").agg(
    last_purchase=("order_date", "max"),
    first_purchase=("order_date", "min"),
    Frequency=("order_id", "nunique"),
    Monetary=("total_amount", "sum")
).reset_index()

sales_rfm["Recency"] = (fecha_ref - sales_rfm["last_purchase"]).dt.days
sales_rfm["Antiguedad"] = (fecha_ref - sales_rfm["first_purchase"]).dt.days

sales_rfm.drop(columns=["last_purchase", "first_purchase"], inplace=True)


In [ ]:
# ===============================
# 3. Variables derivadas de gasto
# ===============================
sales_rfm["Ticket_promedio"] = sales_rfm["Monetary"] / sales_rfm["Frequency"]
sales_rfm["Velocidad_gasto"] = sales_rfm["Monetary"] / (sales_rfm["Recency"] + 1)  # +1 evita división por cero

In [ ]:
# ==============================
# 4. Porcentaje de meses activos
# ==============================
sales_df["year_month"] = sales_df["order_date"].dt.to_period("M")

meses_activos = sales_df.groupby("customer_id")["year_month"].nunique().reset_index()
meses_activos.rename(columns={"year_month": "meses_activos"}, inplace=True)

meses_totales = (fecha_ref.to_period("M") - sales_df.groupby("customer_id")["order_date"].min().dt.to_period("M")).apply(lambda x: x.n).reset_index()
meses_totales.rename(columns={"order_date": "meses_totales"}, inplace=True)

sales_rfm = sales_rfm.merge(meses_activos, on="customer_id").merge(meses_totales, on="customer_id")
sales_rfm["pct_meses_activos"] = sales_rfm["meses_activos"] / (sales_rfm["meses_totales"] + 1)

In [ ]:
# ==========================
# 5. Ratio de repetición
# ==========================
sales_rfm["ratio_repeticion"] = (sales_rfm["Frequency"] - 1) / sales_rfm["Frequency"]

In [ ]:
# =============================================
# 6. Producto favorito y concentración de gasto
# =============================================
cat_gasto = sales_df.groupby(["customer_id", "category"])["total_amount"].sum().reset_index()

# Producto favorito
producto_fav = cat_gasto.loc[cat_gasto.groupby("customer_id")["total_amount"].idxmax()][["customer_id", "category"]]
producto_fav.rename(columns={"category": "producto_favorito"}, inplace=True)

# Concentración de gasto (Herfindahl)
cat_gasto["pct"] = cat_gasto.groupby("customer_id")["total_amount"].transform(lambda x: x / x.sum())
herfindahl = cat_gasto.groupby("customer_id")["pct"].apply(lambda x: (x**2).sum()).reset_index()
herfindahl.rename(columns={"pct": "concentracion_gasto"}, inplace=True)

sales_rfm = sales_rfm.merge(producto_fav, on="customer_id").merge(herfindahl, on="customer_id")

In [ ]:
# =============================
# 7. Porcentaje de devoluciones
# =============================
sales_df["returned_flag"] = sales_df["returned"].apply(lambda x: 1 if str(x).lower() == "yes" else 0)
devoluciones = sales_df.groupby("customer_id")["returned_flag"].mean().reset_index()
devoluciones.rename(columns={"returned_flag": "pct_devoluciones"}, inplace=True)

sales_rfm = sales_rfm.merge(devoluciones, on="customer_id")

In [ ]:
# ==========================
# 8. Margen medio
# ==========================
margen = sales_df.groupby("customer_id")["profit_margin"].mean().reset_index()
margen.rename(columns={"profit_margin": "margen_medio"}, inplace=True)

sales_rfm = sales_rfm.merge(margen, on="customer_id")

print("\nPrimeras filas:")
print(sales_rfm.head())

In [ ]:
# ==========================
# 9. Limpieza de datos
# ==========================

# Log-transformación en variables sesgadas
cols_log = ["Monetary", "Frequency", "Ticket_promedio", "Velocidad_gasto"]
for c in cols_log:
    sales_rfm[c] = np.log1p(sales_rfm[c])  # log(1+x) para evitar log(0)

# Escalar variables numéricas
num_cols = ["Recency", "Frequency", "Monetary", "Ticket_promedio",
            "Velocidad_gasto", "Antiguedad", "pct_meses_activos",
            "ratio_repeticion", "concentracion_gasto",
            "pct_devoluciones", "margen_medio"]

scaler = StandardScaler()
sales_rfm_scaled = scaler.fit_transform(sales_rfm[num_cols])

# One-Hot para producto favorito
sales_rfm_final = pd.get_dummies(sales_rfm[["customer_id", "producto_favorito"]], columns=["producto_favorito"])
sales_rfm_final = sales_rfm_final.merge(pd.DataFrame(sales_rfm_scaled, columns=num_cols), left_index=True, right_index=True)

In [ ]:
# ==========================
# Resultado
# ==========================
print("Dataset final listo para clustering:")
print(sales_rfm_final.head())

---
###4.2. Clustering

- **K-Means**: prueba de 2 a 9 clusters, elige el mejor según - Silhouette.
- **GMM**: igual, pero usando Gaussian Mixture, elige el mejor.
- **DBSCAN**: marca outliers (-1 = ruido).<br>
  Métricas:
  - **Silhouette**: más alto es mejor (cohesión vs separación).
  - **Davies-Bouldin**: más bajo es mejor.
- **Visualización**: reducción con PCA para ver clusters en 2D.
- **Resumen**: imprime métricas y distribución de clusters.
---

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ===========================
# 1. Dataset para clustering
# ===========================
X = sales_rfm_final.drop(columns=["customer_id"])

In [ ]:
# ===========================
# 2. MÉTODO DEL CODO + Silhouette
# ===========================
inertias = []
silhouettes_list = []
davies_list = []
calinski_list = []
k_range = range(2, 16)  # Ampliado hasta 15 clusters

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(X)

    inertias.append(kmeans.inertia_)  # Para método del codo
    silhouettes_list.append(silhouette_score(X, labels))
    davies_list.append(davies_bouldin_score(X, labels))
    calinski_list.append(calinski_harabasz_score(X, labels))

In [ ]:
# Visualizar métricas
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Método del codo
axes[0, 0].plot(k_range, inertias, 'bo-')
axes[0, 0].set_xlabel('Número de clusters (k)')
axes[0, 0].set_ylabel('Inertia (Within-cluster sum of squares)')
axes[0, 0].set_title('Método del Codo')
axes[0, 0].grid(True)

# Silhouette Score (MAYOR es mejor)
axes[0, 1].plot(k_range, silhouettes_list, 'go-')
axes[0, 1].set_xlabel('Número de clusters (k)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Score por k (mayor = mejor)')
axes[0, 1].grid(True)

# Davies-Bouldin (MENOR es mejor)
axes[1, 0].plot(k_range, davies_list, 'ro-')
axes[1, 0].set_xlabel('Número de clusters (k)')
axes[1, 0].set_ylabel('Davies-Bouldin Score')
axes[1, 0].set_title('Davies-Bouldin por k (menor = mejor)')
axes[1, 0].grid(True)

# Calinski-Harabasz (MAYOR es mejor)
axes[1, 1].plot(k_range, calinski_list, 'mo-')
axes[1, 1].set_xlabel('Número de clusters (k)')
axes[1, 1].set_ylabel('Calinski-Harabasz Score')
axes[1, 1].set_title('Calinski-Harabasz por k (mayor = mejor)')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
print("=== RECOMENDACIONES ===")
print(f"Mejor k por Silhouette: {k_range[np.argmax(silhouettes_list)]} (Score: {max(silhouettes_list):.3f})")
print(f"Mejor k por Davies-Bouldin: {k_range[np.argmin(davies_list)]} (Score: {min(davies_list):.3f})")
print(f"Mejor k por Calinski-Harabasz: {k_range[np.argmax(calinski_list)]} (Score: {max(calinski_list):.0f})")

In [ ]:
# ==========================
# 3. SELECCIÓN MANUAL DE K
# ==========================

# Opción 1: Usar el mejor por Silhouette
k_optimo_silhouette = k_range[np.argmax(silhouettes_list)]

# Opción 2: Usar el mejor por Calinski (suele dar más clusters)
k_optimo_calinski = k_range[np.argmax(calinski_list)]

# Opción 3: Forzar un número específico basado en el negocio
k_forzado = 6

print(f"\n=== PROBANDO CON DIFERENTES k ===")
print(f"k óptimo por Silhouette: {k_optimo_silhouette}")
print(f"k óptimo por Calinski: {k_optimo_calinski}")
print(f"k forzado manualmente: {k_forzado}")

In [ ]:
# ==========================
# 4. K-Means con k forzado
# ==========================
kmeans_final = KMeans(n_clusters=k_forzado, random_state=42, n_init=20, max_iter=500)
labels_kmeans = kmeans_final.fit_predict(X)
sales_rfm_final["Cluster_KMeans"] = labels_kmeans

print(f"\nDistribución de clusters K-Means (k={k_forzado}):")
print(sales_rfm_final["Cluster_KMeans"].value_counts().sort_index())

In [ ]:
# ==========================
# 5. GMM con k forzado
# ==========================
gmm_final = GaussianMixture(n_components=k_forzado, covariance_type="full",
                             random_state=42, n_init=10, max_iter=200)
labels_gmm = gmm_final.fit_predict(X)
sales_rfm_final["Cluster_GMM"] = labels_gmm

print(f"\nDistribución de clusters GMM (k={k_forzado}):")
print(sales_rfm_final["Cluster_GMM"].value_counts().sort_index())

In [ ]:
# ==========================
# 6. DBSCAN ajustado
# ==========================
# Para DBSCAN, necesitas ajustar eps según tus datos
from sklearn.neighbors import NearestNeighbors

# Encontrar un eps apropiado
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(X)
distances, indices = neighbors_fit.kneighbors(X)
distances = np.sort(distances[:, -1], axis=0)

plt.figure(figsize=(10, 6))
plt.plot(distances)
plt.xlabel('Puntos ordenados')
plt.ylabel('Distancia al 5to vecino más cercano')
plt.title('Gráfica para elegir eps en DBSCAN (busca el "codo")')
plt.grid(True)
plt.show()

# Prueba varios eps
eps_values = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
for eps in eps_values:
    dbscan = DBSCAN(eps=eps, min_samples=5)
    labels_db = dbscan.fit_predict(X)
    n_clusters = len(set(labels_db)) - (1 if -1 in labels_db else 0)
    n_noise = list(labels_db).count(-1)
    print(f"eps={eps}: {n_clusters} clusters, {n_noise} outliers ({n_noise/len(labels_db)*100:.1f}%)")

eps_final = 1.5
dbscan_final = DBSCAN(eps=eps_final, min_samples=5)
labels_dbscan = dbscan_final.fit_predict(X)
sales_rfm_final["Cluster_DBSCAN"] = labels_dbscan

print(f"\nDistribución de clusters DBSCAN (eps={eps_final}):")
print(sales_rfm_final["Cluster_DBSCAN"].value_counts().sort_index())


In [ ]:
# ==========================
# 7. Visualización mejorada
# ==========================
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
print(f"\nVarianza explicada por PCA: {pca.explained_variance_ratio_[0]:.2%} (PC1), {pca.explained_variance_ratio_[1]:.2%} (PC2)")
print(f"Varianza total explicada: {sum(pca.explained_variance_ratio_):.2%}")

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# K-Means
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=labels_kmeans,
                           cmap='tab10', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0].set_title(f'K-Means ({k_forzado} clusters) - PCA 2D')
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} var)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} var)')
plt.colorbar(scatter1, ax=axes[0])

# GMM
scatter2 = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=labels_gmm,
                           cmap='tab10', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1].set_title(f'GMM ({k_forzado} clusters) - PCA 2D')
axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} var)')
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} var)')
plt.colorbar(scatter2, ax=axes[1])

# DBSCAN
scatter3 = axes[2].scatter(X_pca[:, 0], X_pca[:, 1], c=labels_dbscan,
                           cmap='tab10', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[2].set_title(f'DBSCAN (eps={eps_final}) - PCA 2D')
axes[2].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} var)')
axes[2].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} var)')
plt.colorbar(scatter3, ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
# ==========================
# 8. DIAGNÓSTICO ADICIONAL
# ==========================
print("\n=== DIAGNÓSTICO: ¿Por qué pocos clusters? ===")
print(f"Número de clientes: {len(X)}")
print(f"Número de variables: {len(X.columns)}")
print(f"Varianza explicada por 2 componentes PCA: {sum(pca.explained_variance_ratio_):.2%}")

if sum(pca.explained_variance_ratio_) > 0.80:
    print("Las primeras 2 componentes explican >80% varianza: datos muy comprimibles")

# Correlaciones entre variables
print("\nVariables más correlacionadas (pueden redundar):")
corr_matrix = sales_rfm_final[["Recency", "Frequency", "Monetary", "Ticket_promedio",
                                 "Velocidad_gasto", "Antiguedad"]].corr()
high_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

for v1, v2, corr in high_corr:
    print(f"  {v1} vs {v2}: {corr:.3f}")

if not high_corr:
    print("  No hay correlaciones altas detectadas")

In [ ]:
# ==========================
# 9. Interpretación de clusters
# ==========================

# Variables de negocio a analizar por cluster
vars_negocio = [
    "Recency", "Frequency", "Monetary", "Ticket_promedio", "Velocidad_gasto",
    "Antiguedad", "pct_meses_activos", "ratio_repeticion",
    "concentracion_gasto", "pct_devoluciones", "margen_medio"
]

def resumir_clusters(df, cluster_col):
    resumen = df.groupby(cluster_col)[vars_negocio].mean().round(2)
    resumen["count_clientes"] = df.groupby(cluster_col).size()
    return resumen.sort_values("Monetary", ascending=False)

# Resumen KMeans
print("=== Perfil de Clusters K-Means ===")
print(resumir_clusters(sales_rfm_final, "Cluster_KMeans"))

# Resumen GMM
print("\n=== Perfil de Clusters GMM ===")
print(resumir_clusters(sales_rfm_final, "Cluster_GMM"))

# Resumen DBSCAN (solo para clusters, ignora -1 si quieres ver solo clientes válidos)
print("\n=== Perfil de Clusters DBSCAN ===")
print(resumir_clusters(sales_rfm_final, "Cluster_DBSCAN"))

---
###4.3. Etiquetas de los clusters
---

In [ ]:
import pandas as pd
import numpy as np

def etiquetar_clusters_mejorado(df, cluster_col, verbose=True):
    """
    Sistema de etiquetado basado en percentiles y lógica de negocio clara.
    Garantiza etiquetas únicas y muestra el razonamiento.
    """

    # Calcular estadísticas por cluster
    resumen = df.groupby(cluster_col)[vars_negocio].mean()
    conteos = df[cluster_col].value_counts().sort_index()

    # Definir umbrales basados en percentiles
    umbrales = {
        'Recency_muy_bajo': df['Recency'].quantile(0.25),    # Compraron muy recientemente
        'Recency_bajo': df['Recency'].quantile(0.40),
        'Recency_alto': df['Recency'].quantile(0.60),
        'Recency_muy_alto': df['Recency'].quantile(0.75),    # Hace mucho no compran

        'Frequency_muy_baja': df['Frequency'].quantile(0.25),
        'Frequency_baja': df['Frequency'].quantile(0.40),
        'Frequency_media': df['Frequency'].quantile(0.60),
        'Frequency_alta': df['Frequency'].quantile(0.75),

        'Monetary_muy_bajo': df['Monetary'].quantile(0.25),
        'Monetary_bajo': df['Monetary'].quantile(0.40),
        'Monetary_medio': df['Monetary'].quantile(0.60),
        'Monetary_alto': df['Monetary'].quantile(0.75),

        'Margen_bajo': df['margen_medio'].quantile(0.33),
        'Margen_alto': df['margen_medio'].quantile(0.67),

        'Ticket_bajo': df['Ticket_promedio'].quantile(0.33),
        'Ticket_alto': df['Ticket_promedio'].quantile(0.67),

        'Devoluciones_alto': df['pct_devoluciones'].quantile(0.75),
        'Repeticion_baja': df['ratio_repeticion'].quantile(0.33),
        'Velocidad_alta': df['Velocidad_gasto'].quantile(0.75),
    }

    def clasificar_cluster(row, cluster_id):
        """
        Clasifica un cluster basado en reglas de negocio claras y priorizadas.
        Retorna (etiqueta, razón, prioridad)
        """
        R = row['Recency']
        F = row['Frequency']
        M = row['Monetary']
        margen = row['margen_medio']
        ticket = row['Ticket_promedio']
        devol = row['pct_devoluciones']
        repet = row['ratio_repeticion']
        veloc = row['Velocidad_gasto']

        razones = []

        # NIVEL 1: VIP / Champions (Prioridad 10)
        if (M > umbrales['Monetary_alto'] and
            F > umbrales['Frequency_alta'] and
            R < umbrales['Recency_bajo'] and
            margen > umbrales['Margen_bajo']):
            return ('VIP / Champions',
                    f"Alto gasto (M={M:.2f}), alta frecuencia (F={F:.2f}), reciente (R={R:.2f}), buen margen ({margen:.2f})",
                    10)

        # NIVEL 2: Big Spenders con Alto Margen (Prioridad 9)
        if (M > umbrales['Monetary_alto'] and
            margen > umbrales['Margen_alto'] and
            F > umbrales['Frequency_baja']):
            return ('Big Spenders Alto Margen',
                    f"Muy alto gasto (M={M:.2f}), excelente margen ({margen:.2f})",
                    9)

        # NIVEL 3: Big Spenders con Bajo Margen (Prioridad 8)
        if (M > umbrales['Monetary_alto'] and
            margen < umbrales['Margen_bajo']):
            return ('Big Spenders Bajo Margen',
                    f"Alto gasto (M={M:.2f}), pero bajo margen ({margen:.2f})",
                    8)

        # NIVEL 4: Clientes Leales (Prioridad 7)
        if (F > umbrales['Frequency_alta'] and
            repet > umbrales['Repeticion_baja'] and
            R < umbrales['Recency_alto']):
            return ('Clientes Leales',
                    f"Alta frecuencia (F={F:.2f}), buena repetición ({repet:.2f}), activos (R={R:.2f})",
                    7)

        # NIVEL 5: Potenciales Leales / Regulares (Prioridad 6)
        if (F > umbrales['Frequency_baja'] and
            M > umbrales['Monetary_bajo'] and
            R < umbrales['Recency_alto']):
            return ('Clientes Regulares',
                    f"Frecuencia media-alta (F={F:.2f}), gasto medio (M={M:.2f}), activos (R={R:.2f})",
                    6)

        # NIVEL 6: Sprinters / Alta Velocidad (Prioridad 5)
        if veloc > umbrales['Velocidad_alta'] and R < umbrales['Recency_bajo']:
            return ('Sprinters (alta velocidad)',
                    f"Alta velocidad de gasto ({veloc:.2f}), muy recientes (R={R:.2f})",
                    5)

        # NIVEL 7: Nuevos Clientes (Prioridad 4)
        if F <= umbrales['Frequency_muy_baja'] and R < umbrales['Recency_bajo']:
            return ('Nuevos Clientes',
                    f"Baja frecuencia (F={F:.2f}), pero recientes (R={R:.2f}) - nuevos",
                    4)

        # NIVEL 8: En Riesgo (Prioridad 3)
        if (R > umbrales['Recency_alto'] and
            R < umbrales['Recency_muy_alto'] and
            M > umbrales['Monetary_bajo'] and
            F > umbrales['Frequency_baja']):
            return ('Clientes en Riesgo',
                    f"Inactivos (R={R:.2f}), pero antes compraban (F={F:.2f}, M={M:.2f})",
                    3)

        # NIVEL 9: Hibernando (Prioridad 2)
        if R > umbrales['Recency_muy_alto'] and F > umbrales['Frequency_baja']:
            return ('Hibernando',
                    f"Muy inactivos (R={R:.2f}), pero con historial (F={F:.2f})",
                    2)

        # NIVEL 10: Perdidos (Prioridad 1)
        if R > umbrales['Recency_muy_alto'] and F <= umbrales['Frequency_muy_baja']:
            return ('Perdidos',
                    f"Muy inactivos (R={R:.2f}), baja frecuencia (F={F:.2f})",
                    1)

        # CASOS ESPECIALES

        # Cazadores de Ofertas
        if devol > umbrales['Devoluciones_alto'] and margen < umbrales['Margen_bajo']:
            return ('Cazadores de Ofertas',
                    f"Altas devoluciones ({devol:.2f}), bajo margen ({margen:.2f})",
                    4)

        # Compradores Ocasionales (por defecto para bajo F)
        if F <= umbrales['Frequency_muy_baja']:
            return ('Compradores Ocasionales',
                    f"Muy baja frecuencia (F={F:.2f}), uso esporádico",
                    3)

        # Bajo Valor (por defecto para M y F bajos)
        if M < umbrales['Monetary_bajo'] and F < umbrales['Frequency_media']:
            return ('Bajo Valor',
                    f"Bajo gasto (M={M:.2f}), baja frecuencia (F={F:.2f})",
                    2)

        # Por defecto: Clientes Estándar
        return ('Clientes Estándar',
                f"Perfil promedio - R={R:.2f}, F={F:.2f}, M={M:.2f}",
                5)

    # Clasificar todos los clusters
    clasificaciones = {}
    for cluster, row in resumen.iterrows():
        etiqueta, razon, prioridad = clasificar_cluster(row, cluster)
        clasificaciones[cluster] = {
            'etiqueta': etiqueta,
            'razon': razon,
            'prioridad': prioridad
        }

    # Resolver conflictos (etiquetas duplicadas)
    etiquetas_finales = {}
    etiquetas_usadas = set()

    # Ordenar por prioridad (mayor prioridad elige primero)
    clusters_ordenados = sorted(
        clasificaciones.keys(),
        key=lambda c: clasificaciones[c]['prioridad'],
        reverse=True
    )

    for cluster in clusters_ordenados:
        etiq_original = clasificaciones[cluster]['etiqueta']

        if etiq_original not in etiquetas_usadas:
            etiquetas_finales[cluster] = etiq_original
            etiquetas_usadas.add(etiq_original)
        else:
            # Si está usada, buscar alternativa
            alternativas = [
                'Clientes Estándar',
                'Clientes Regulares',
                'Compradores Moderados',
                'Clientes Promedio',
                'Segmento Mixto',
                'Otros Clientes'
            ]
            for alt in alternativas:
                if alt not in etiquetas_usadas:
                    etiquetas_finales[cluster] = alt
                    etiquetas_usadas.add(alt)
                    clasificaciones[cluster]['razon'] += f" [Renombrado a '{alt}' por duplicado]"
                    break

    # Crear DataFrame de resultados
    resultado_df = pd.DataFrame({
        'Cluster': list(etiquetas_finales.keys()),
        'Etiqueta': list(etiquetas_finales.values()),
        'Cantidad': [conteos[c] for c in etiquetas_finales.keys()],
        'Porcentaje': [f"{(conteos[c]/len(df)*100):.1f}%" for c in etiquetas_finales.keys()],
        'Recency': [resumen.loc[c, 'Recency'] for c in etiquetas_finales.keys()],
        'Frequency': [resumen.loc[c, 'Frequency'] for c in etiquetas_finales.keys()],
        'Monetary': [resumen.loc[c, 'Monetary'] for c in etiquetas_finales.keys()],
        'Margen': [resumen.loc[c, 'margen_medio'] for c in etiquetas_finales.keys()],
        'Razón': [clasificaciones[c]['razon'] for c in etiquetas_finales.keys()]
    })

    # Ordenar por cantidad (más grande primero)
    resultado_df = resultado_df.sort_values('Cantidad', ascending=False)

    # Aplicar etiquetas al DataFrame original
    df[f'{cluster_col}_Etiqueta'] = df[cluster_col].map(etiquetas_finales)

    if verbose:
        print(f"\n{'='*80}")
        print(f"DETALLE DE CLASIFICACIÓN - {cluster_col}")
        print(f"{'='*80}\n")
        for _, row in resultado_df.iterrows():
            print(f"Cluster {row['Cluster']}: {row['Etiqueta']}")
            print(f"   {row['Cantidad']} clientes ({row['Porcentaje']})")
            print(f"   R={row['Recency']:.2f} | F={row['Frequency']:.2f} | M={row['Monetary']:.2f} | Margen={row['Margen']:.2f}")
            print(f"   {row['Razón']}")
            print()

    return df, resultado_df, etiquetas_finales


In [ ]:
# ============================================
# APLICAR A LOS TRES MODELOS
# ============================================

print("\n" + "=" * 80 )
print("ANÁLISIS Y ETIQUETADO DE CLUSTERS RFM")
print( "=" * 80 +"\n")

print("=" * 80)
print("K-MEANS")
print("=" * 80)
sales_rfm_final, resultado_kmeans, etiq_kmeans = etiquetar_clusters_mejorado(
    sales_rfm_final, 'Cluster_KMeans', verbose=True
)

print("\n" + "=" * 80)
print("GAUSSIAN MIXTURE MODEL (GMM)")
print("=" * 80)
sales_rfm_final, resultado_gmm, etiq_gmm = etiquetar_clusters_mejorado(
    sales_rfm_final, 'Cluster_GMM', verbose=True
)

print("\n" + "=" * 80)
print("DBSCAN")
print("=" * 80)
sales_rfm_final, resultado_dbscan, etiq_dbscan = etiquetar_clusters_mejorado(
    sales_rfm_final, 'Cluster_DBSCAN', verbose=True
)

# ============================================
# RESUMEN COMPARATIVO
# ============================================

print("\n" + "=" * 80)
print("RESUMEN COMPARATIVO DE MODELOS")
print("=" * 80 + "\n")

def resumen_modelo(resultado_df, nombre):
    print(f"🔹 {nombre}:")
    print(f"   Clusters encontrados: {len(resultado_df)}")
    print(f"   Etiquetas únicas: {resultado_df['Etiqueta'].nunique()}")
    print(f"   Cluster más grande: {resultado_df.iloc[0]['Etiqueta']} ({resultado_df.iloc[0]['Porcentaje']})")
    print(f"   Cluster más pequeño: {resultado_df.iloc[-1]['Etiqueta']} ({resultado_df.iloc[-1]['Porcentaje']})")
    print()

resumen_modelo(resultado_kmeans, "K-Means")
resumen_modelo(resultado_gmm, "GMM")
resumen_modelo(resultado_dbscan, "DBSCAN")

In [ ]:
# ============================================
# TABLA COMPARATIVA SIMPLIFICADA
# ============================================

print("=" * 80)
print("TABLA RESUMEN - K-MEANS")
print("=" * 80)
print(resultado_kmeans[['Cluster', 'Etiqueta', 'Cantidad', 'Porcentaje']].to_string(index=False))

print("\n" + "=" * 80)
print("TABLA RESUMEN - GMM")
print("=" * 80)
print(resultado_gmm[['Cluster', 'Etiqueta', 'Cantidad', 'Porcentaje']].to_string(index=False))

print("\n" + "=" * 80)
print("TABLA RESUMEN - DBSCAN")
print("=" * 80)
print(resultado_dbscan[['Cluster', 'Etiqueta', 'Cantidad', 'Porcentaje']].to_string(index=False))

print("\nEtiquetado completado exitosamente!")
print("Columnas creadas:")
print("   • Cluster_KMeans_Etiqueta")
print("   • Cluster_GMM_Etiqueta")
print("   • Cluster_DBSCAN_Etiqueta")

---
###4.4. Representacion visual de los clusters
---


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================
# 1. Heatmap de promedios normalizados por cluster
# ==========================
def plot_heatmap(df, cluster_col):
    resumen = df.groupby(cluster_col)[vars_negocio].mean()
    # Normalizar cada columna 0–1 para comparabilidad
    resumen_norm = (resumen - resumen.min()) / (resumen.max() - resumen.min())

    plt.figure(figsize=(12,6))
    sns.heatmap(resumen_norm, annot=True, cmap="Blues", fmt=".2f", cbar=True)
    plt.title(f"Perfil normalizado de {cluster_col}", fontsize=14)
    plt.ylabel("Cluster")
    plt.show()

# ==========================
# 2. Boxplots por variable y cluster
# ==========================
def plot_boxplots(df, cluster_col, variables):
    n = len(variables)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 5))

    if n == 1:
        axes = [axes]

    for ax, var in zip(axes, variables):
        sns.boxplot(x=cluster_col, y=var, data=df, ax=ax, palette="Set3")
        ax.set_title(f"{var} por cluster")
        ax.set_xlabel("Cluster")
        ax.set_ylabel(var)

    plt.tight_layout()
    plt.show()

# ==========================
# 3. Ejecutar reportes
# ==========================
# Heatmap para KMeans
plot_heatmap(sales_rfm_final, "Cluster_KMeans_Etiqueta")

# Boxplots de variables clave para KMeans
plot_boxplots(sales_rfm_final, "Cluster_KMeans_Etiqueta", ["Recency", "Frequency", "Monetary", "Ticket_promedio", "pct_devoluciones"])


---
###4.5. Reporte de la clasificacion
---

In [ ]:
def generar_reporte_segmentos(df, cluster_col):
    resumen = df.groupby(cluster_col)[vars_negocio].mean().round(2)
    reporte = []

    for cluster, row in resumen.iterrows():
        desc = f"Segmento {cluster} ({cluster_col}):\n"

        # Recency
        if row["Recency"] < resumen["Recency"].median():
            desc += "- Clientes recientes (compraron hace poco).\n"
        else:
            desc += "- Clientes inactivos (última compra hace tiempo).\n"

        # Frequency
        if row["Frequency"] > resumen["Frequency"].median():
            desc += "- Compran con frecuencia.\n"
        else:
            desc += "- Compran rara vez.\n"

        # Monetary
        if row["Monetary"] > resumen["Monetary"].median():
            desc += "- Gastan por encima del promedio.\n"
        else:
            desc += "- Gastan por debajo del promedio.\n"

        # Ticket promedio
        if row["Ticket_promedio"] > resumen["Ticket_promedio"].median():
            desc += "- Ticket promedio alto (compras grandes).\n"
        else:
            desc += "- Ticket promedio bajo (compras pequeñas).\n"

        # Velocidad de gasto
        if row["Velocidad_gasto"] > resumen["Velocidad_gasto"].median():
            desc += "- Alta velocidad de gasto (aportan rápido).\n"

        # % meses activos
        if row["pct_meses_activos"] > 0.5:
            desc += "- Se mantienen activos la mayor parte del tiempo.\n"
        else:
            desc += "- Activos en pocos meses.\n"

        # Ratio repetición
        if row["ratio_repeticion"] < 0.5:
            desc += "- Mayoría son compradores de una sola vez.\n"

        # % devoluciones
        if row["pct_devoluciones"] > 0.2:
            desc += "- Alta tasa de devoluciones, clientes problemáticos.\n"
        else:
            desc += "- Baja tasa de devoluciones.\n"

        # Margen
        if row["margen_medio"] > resumen["margen_medio"].median():
            desc += "- Compran productos de buen margen.\n"
        else:
            desc += "- Suelen comprar con márgenes bajos.\n"

        # Concentración
        if row["concentracion_gasto"] > 0.7:
            desc += "- Especialistas: concentran gasto en pocas categorías.\n"
        else:
            desc += "- Diversificados: compran en varias categorías.\n"

        desc += "\n"
        reporte.append(desc)

    return "\n".join(reporte)

# Generar reporte para KMeans con etiquetas
print("=== Reporte narrativo KMeans ===\n")
print(generar_reporte_segmentos(sales_rfm_final, "Cluster_KMeans_Etiqueta"))

print("\n=== Reporte narrativo GMM ===\n")
print(generar_reporte_segmentos(sales_rfm_final, "Cluster_GMM_Etiqueta"))

print("\n=== Reporte narrativo DBSCAN ===\n")
print(generar_reporte_segmentos(sales_rfm_final, "Cluster_DBSCAN_Etiqueta"))
